# CRA IP FY17 Q1 

In [1]:
import pandas as pd
from xlwings import Workbook, Range
import numpy as np
import os, glob, collections, functools, itertools, re, datetime

In [2]:
quarterStart = pd.Timestamp('2016-7-1')#start of the quarter
quarterEnd = pd.Timestamp('2016-9-30')#end of the quarter
asOnDate = '24-October-2016'

In [3]:
def headerfiller(df):
    df.columns = [re.sub(r' |\n', '_', c) for c in df.columns]

In [4]:
path_cancelled = r"C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\IMPACT\RP76_On Site_Remote Visit Report Review Metrics_Ver-Oct2014 Cancelled Stopped.xls"
path_delayed = r"C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\IMPACT\RP76_On Site_Remote Visit Report Review Metrics_Ver-Oct2014 delayed.xls"
path_ongoing = r"C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\IMPACT\RP76_On Site_Remote Visit Report Review Metrics_Ver-Oct2014 Ongoing.xlsx"
path_endOfQuarterER = r"C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\Oracle\Employee Report 24-Oct-2016.xlsx"
path_asOfDateER = "C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\Oracle\Oracle Employee Report 17-Oct-2016.xlsx"

#as on end of quarter
#path_shr = 

path_jhr = r"C:\Users\ReddyAl\Documents\CRA\FY17 Q1\Data from HCM and BI\Oracle\Job History Report 01-Apr-2016 to 30-Sep-2016.csv"

In [5]:
cancelled = pd.read_excel(path_cancelled, sheetname='Visit Details')#, parse_dates = True)
delayed = pd.read_excel(path_delayed, sheetname='Visit Details')#, parse_dates = True)
ongoing = pd.read_excel(path_ongoing, sheetname='Visit Details')#, parse_dates = True)

## 30-Sep-2016 - End of quarter 2017 Q1

In [6]:
endOfQuarterER = pd.read_excel(path_endOfQuarterER)

In [7]:
#27-July-2016

In [8]:
asOfDateER = pd.read_excel(path_asOfDateER)

#Supervisor hierarchy report as on EoQ
#shr = pd.read_excel(r"C:\Users\ReddyAl\Documents\CRA\FY16 Q4\Data from HCM and BI\HCM\Supervisor Hierarchy Report 30-June-2016.xlsx")

#Job History report from 01-Apr-2016 to 30-Sep-2016
jhr = pd.read_csv(path_jhr)

In [9]:
headerfiller(endOfQuarterER)
headerfiller(asOfDateER)
#headerfiller(shr)
headerfiller(jhr)
headerfiller(ongoing)
headerfiller(delayed)
headerfiller(cancelled)

In [10]:
#shr.rename(columns={'Employee_Number':'PID'}, inplace=True)
jhr.rename(columns={'Employee_Number':'PID'}, inplace=True)

In [11]:
ongoing.loc[ongoing.Monitor_ID.isnull(), 'PID'] = ongoing['Additional_Monitor_PID']
ongoing.loc[~ongoing.Monitor_ID.isnull(), 'PID'] = ongoing['Monitor_ID']
delayed.loc[delayed.Monitor_ID.isnull(), 'PID'] = delayed['Additional_Monitor_PID']
delayed.loc[~delayed.Monitor_ID.isnull(), 'PID'] = delayed['Monitor_ID']
cancelled.loc[cancelled.Monitor_ID.isnull(), 'PID'] = cancelled['Additional_Monitor_PID']
cancelled.loc[~cancelled.Monitor_ID.isnull(), 'PID'] = cancelled['Monitor_ID']

ongoing.loc[ongoing.Monitor.isnull(), 'CRA'] = ongoing['Additional_Monitor_Name']
ongoing.loc[~ongoing.Monitor.isnull(), 'CRA'] = ongoing['Monitor']
delayed.loc[delayed.Monitor.isnull(), 'CRA'] = delayed['Additional_Monitor_Name']
delayed.loc[~delayed.Monitor.isnull(), 'CRA'] = delayed['Monitor']
cancelled.loc[cancelled.Monitor.isnull(), 'CRA'] = cancelled['Additional_Monitor_Name']
cancelled.loc[~cancelled.Monitor.isnull(), 'CRA'] = cancelled['Monitor']

In [12]:
vists_list = [ongoing, delayed, cancelled]
visits = pd.concat(vists_list)

In [13]:
visits = visits[pd.notnull(visits['Visit_Start_Date'])]
#visits['Visit_Start_Date'] = visits.Visit_Start_Date.astype('str')
visits['Visit_Start_Date'] = pd.to_datetime(visits['Visit_Start_Date'])
visits['Visit_End_Date'] = pd.to_datetime(visits['Visit_End_Date'])
mask = (visits['Visit_Start_Date'] >= quarterStart) & (visits['Visit_Start_Date'] <= quarterEnd)
visits = visits.loc[mask]


In [14]:
visits['StartMonth'] = pd.DatetimeIndex(visits['Visit_Start_Date']).month
visits['StartYear'] = pd.DatetimeIndex(visits['Visit_Start_Date']).year

In [15]:
ConsolidatedVisits = pd.DataFrame(visits.pivot_table(values = 'CRA', columns=['StartMonth'], index=['PID'], aggfunc=np.count_nonzero)).reset_index()

In [16]:
endOfQuarterER.rename(columns={'Employee_Full_Name':"Employee_Full_Name0"
                   ,'Job_Title_Description':'Job_Title_Description0'
                   ,'Supervisor_Employee_Number':'Supervisor_Employee_Number0',
                    'Employee_Number':'PID'}, inplace = True)

for i in range(16):
    for col in ['Employee_Full_Name','Job_Title_Description','Supervisor_Employee_Number']:
        endOfQuarterER[col+str(i+1)] = endOfQuarterER['Supervisor_Employee_Number'+str(i)].map( 
                                            endOfQuarterER.set_index('PID')[col+'0'] )

endOfQuarterER.rename(columns={'Employee_Full_Name0':"Employee_Full_Name"
                   ,'Job_Title_Description0':'Job_Title_Description'
                   ,'Supervisor_Employee_Number0':'Supervisor_Employee_Number'}, inplace = True)

In [17]:
visitDataWEndoFQuarterER = ConsolidatedVisits.merge(endOfQuarterER, on='PID', how='left')

In [18]:
visitDataWEndoFQuarterER['Payroll_Country'] = visitDataWEndoFQuarterER.Payroll_Country.str.lower()

In [19]:
countryData = pd.read_excel(r"C:\Users\ReddyAl\Documents\CRA\FY16 Q4\Country Data with Visit Threshold, Per Visit, Std Hrs and FxRates.xlsx")

countryData.rename(columns={'Country':'Payroll_Country'}, inplace=True)
countryData['Payroll_Country'] = countryData.Payroll_Country.str.lower()

visitERCountry = visitDataWEndoFQuarterER.merge(countryData, on='Payroll_Country', how='left')

In [20]:
jhr['StartMonth'] = pd.DatetimeIndex(jhr['DW_Effective_Start_Date']).month
jhr['StartYear'] = pd.DatetimeIndex(jhr['DW_Effective_Start_Date']).year
jhr['Counts'] = jhr.groupby("PID")['PID'].transform('count')

In [21]:
df = pd.melt(jhr, id_vars=['PID','Counts','StartMonth'], value_vars=['Job_Title_Description'], value_name='Job_Title_Description')
jobHistory = pd.DataFrame(df.pivot_table(index=['PID'], columns='StartMonth', values='Job_Title_Description', aggfunc = lambda x: ', '.join(x))).reset_index()

In [22]:
#writer = pd.ExcelWriter('job history.xlsx')
#jobHistory.to_excel(writer, 'Job History')
#writer.save()

In [23]:
visitERCountryJH = visitERCountry.merge(jobHistory, on='PID', how='left')
visitERCountryJH['Termination_Status ' + asOnDate] = "No"
visitERCountryJH.loc[~visitERCountryJH.PID.isin(asOfDateER['PID'].tolist()), ('Termination_Status' + asOnDate)] = 'Yes'
writer = pd.ExcelWriter('FY17 Q1 raw master.xlsx')
visitERCountryJH.to_excel(writer, 'Raw Master')
writer.save()